In [ ]:
url = "https://www.ica.se/erbjudanden/ica-aspudden-1003601/"

service = get_gmail_service()
msg = create_message(sender="johnavigdor@gmail.com",
                      to="johnavigdor@gmail.com",
                      subject="Testing Gmail API",
                      message_text="This is a test email from gmail API")

send_message(service,"me",msg)

In [15]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def create_recipe_chain(model_name: str = "gpt-4o", temperature: float = 0.2):
    """
    Initializes and returns a SequentialChain that generates recipes
    from grocery offers and compiles a shopping list.

    Args:
        model_name (str): OpenAI model name.
        temperature (float): LLM temperature.

    Returns:
        function: A function that can be called with offers_text to get recipes and shopping list.
    """
    llm = ChatOpenAI(model=model_name, temperature=temperature, openai_api_key=OPENAI_API_KEY)

    recipe_prompt = ChatPromptTemplate.from_template("""
                                        
        You are a helpful cooking assistant.
        You receive a list of grocery offers and their prices from a local supermarket. It looks like this:

        {offers}

        Using **only** the ingredients or appropriate transformations of the ingredients. You may add common pantry items 
        like salt, pepper, oil, rice, potatoes, pasta. 

        Your task is to generate **three distinct recipes**.
        For each recipe, provide:

        - Recipe name
        - Ingredients (with quantities in grams, liters, or pieces)
        - Step-by-step cooking instructions

        Output in a structured way:

        Recipe 1: ...
        Ingredients:
        -...
        Instructions:
        1. ...
        2. ...

        Recipe 2: ...
        ... etc.
        """
    )
    chain_recipes = LLMChain(
        llm=llm,
        prompt=recipe_prompt,
        output_key="recipes",
        verbose=True
    )

    # Chain B, from recipes to shopping list
    shopping_list_prompt = ChatPromptTemplate.from_template(
        """
        You are a helpful assistant.
        You receive **three recipes** in the following structured format:

        {recipes}

        Your task is to generate a **compiled shopping list** of all ingredients needed for the three recipes.
        Combine duplicates (e.g. if "tomatoes" appear in multiple recipes, sum the quantities).
        List each ingredient and the total quantity (with units) needed.
        
        Output **only** the shopping list, one ingredient per line, in a readable format"""

    )

    chain_shopping_list = LLMChain(
        llm=llm,
        prompt=shopping_list_prompt,
        output_key="shopping_list",
        verbose=True
    )

    # Sequential chain
    overall_chain = SequentialChain(
        chains=[chain_recipes, chain_shopping_list],
        input_variables=["offers"],
        output_variables=["recipes", "shopping_list"],
        verbose=True
    )

    def run_chain(offers_text: str):
        return overall_chain.invoke({"offers": offers_text})
    
    return run_chain

In [17]:
from web_scraper import scrape_ica_offers

url = "https://www.ica.se/erbjudanden/ica-supermarket-aptiten-1003988/"
offers = scrape_ica_offers(url)

offers_text = "\n".join([f"{title}: {price}" for title, price in offers])

recipe_chain = create_recipe_chain(model_name="gpt-4o", temperature=0.2)
output = recipe_chain(offers_text)

recipes_text = output["recipes"]
shopping_list_text = output["shopping_list"]













> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 

        You are a helpful cooking assistant.
        You receive a list of grocery offers and their prices from a local supermarket. It looks like this:

        Toalettpapper, hushållspapperICA Skona: 4 för 100 kr
Rosor 9-pack, AzaleaICA: 69 kr/st
Avokado i nätICA: 35 kr/st
Färsk laxsidaICA Gott liv: 149 kr/kg
Fryst mangoICA: 2 för 35 kr
KronljusICA: 2 för 119 kr
LäskTrocadero, Loka Crush, Champis: 2 för 30 kr
DuschcremeFamily Fresh: 2 för 32 kr
JuiceTropicana: 29 kr/+pant 1:-
KebabSchysst käk: 3 för 99 kr
LäskPepsi: 2 för 65 kr
MandelmassaICA: 20 kr/st
FläskfiléICA: 169 kr/kg
KasslerICA: 79,90 kr/kg
Nötstek av fransyskaICA: 119 kr/kg
Oxfilé i bitICA: 429 kr/kg
CitronerICA: 4 för 20 kr
Gröna kärnfria druvor i askICA: 25 kr/st
Röd spetspaprika i påseICA: 15 kr/st
Äpplen Royal GalaICA: 25 kr/kg
Brysselkål i påseICA: 20 kr/st
Ekologiska snackmorötter i påseICA I love eco: 2 för 

In [18]:
print(recipes_text)

Recipe 1: Avocado and Smoked Salmon Salad

Ingredients:
- 2 Avocados (from Avokado i nät)
- 200g Fresh Salmon (from Färsk laxsidaICA Gott liv)
- 1 Lemon (from CitronerICA)
- 100g Green Seedless Grapes (from Gröna kärnfria druvor i askICA)
- 50g Arugula (common pantry item)
- Salt and pepper (common pantry items)
- Olive oil (common pantry item)

Instructions:
1. Slice the avocados in half, remove the pit, and scoop out the flesh. Cut into thin slices.
2. Thinly slice the fresh salmon into strips.
3. Halve the grapes.
4. In a large bowl, combine the avocado slices, salmon strips, and halved grapes.
5. Squeeze the juice of one lemon over the salad.
6. Add arugula to the bowl.
7. Drizzle with olive oil and season with salt and pepper to taste.
8. Toss gently to combine and serve immediately.

Recipe 2: Chicken and Bacon Pasta

Ingredients:
- 200g Chicken Nuggets (from Kycklingburgare, Nuggets, WingsKronfågel/MAX)
- 100g Bacon (from BaconScan)
- 250g Pasta (common pantry item)
- 1 Red Bell

In [19]:
print(shopping_list_text)

- 2 Avocados
- 200g Fresh Salmon
- 1 Lemon
- 100g Green Seedless Grapes
- 50g Arugula
- Salt (common pantry item)
- Pepper (common pantry item)
- Olive oil (common pantry item)
- 200g Chicken Nuggets
- 100g Bacon
- 250g Pasta
- 1 Red Bell Pepper
- 1 Onion
- 2 cloves Garlic
- 100ml Cream
- 3 Royal Gala Apples
- 1 pack Puff Pastry
- 50g Butter
- 50g Sugar
- 1 tsp Cinnamon
- 1 Egg
